In [1]:
# Import libraries
# try:
  # %tensorflow_version only exists in Colab.
#    !pip install tf-nightly
# except Exception:
#    pass

import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.17.0-dev20240303


In [2]:
# Retrieve data files
# !wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
# !wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
# Creating dataframes from tsv files
df_train = pd.read_csv(train_file_path, sep = '\t', header = None, names = ['label', 'text'])
df_test = pd.read_csv(test_file_path, sep = '\t', header = None, names = ['label', 'text'])

In [4]:
df_train.head()

,label,text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [5]:
df_test.head()

,label,text
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [6]:
# Printing number of rows from each dataframe
print(len(df_train))
print(len(df_test))

4179
1392


In [7]:
# Converting labels to integer
df_train['label'] = df_train['label'].replace(to_replace = {'ham': 0, 'spam': 1})
df_test['label'] = df_test['label'].replace(to_replace = {'ham': 0, 'spam': 1})

In [8]:
df_train['label'][0]

0

In [9]:
df_test['label'][0]

0

In [10]:
# Determining length of longest text (df_train)
train_msg_len = df_train['text'].apply(len)
print(max(train_msg_len))

910


In [11]:
# Determining length of longest text (df_test)
test_msg_len = df_test['text'].apply(len)
print(max(test_msg_len))

482


In [12]:
# Set parameters for text preprocessing and model training
MAXLEN = 250
BATCH_SIZE = 128

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

# Initialize the tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the training text data
tokenizer.fit_on_texts(df_train['text'])

# Calculate the vocabulary size based on the tokenizer's word index
VOCAB_SIZE = len(tokenizer.word_index)

# Convert text to sequences of integers
train_sequences = tokenizer.texts_to_sequences(df_train['text'])
test_sequences = tokenizer.texts_to_sequences(df_test['text'])

# Pad sequences
train_sequences_padded = sequence.pad_sequences(train_sequences, maxlen = MAXLEN)
test_sequences_padded = sequence.pad_sequences(test_sequences, maxlen = MAXLEN)

In [13]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, BATCH_SIZE, input_shape = (MAXLEN,)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences = True)),
    tf.keras.layers.LSTM(64, dropout = 0.2, recurrent_dropout = 0.2),
    tf.keras.layers.Dense(1, activation = "sigmoid")])

C:\Users\pcdud\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
# Compile the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# View model summary
model.summary()

# Train the model
history = model.fit(train_sequences_padded, df_train['label'], 
                    batch_size = BATCH_SIZE, 
                    epochs = 10, 
                    validation_split = 0.2)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 250, 128)       │       984,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 250, 128)       │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,132,865 (4.32 MB)

 Trainable params: 1,132,865 (4.32 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 632ms/step - accuracy: 0.8182 - loss: 0.4492 - val_accuracy: 0.9701 - val_loss: 0.1150
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 628ms/step - accuracy: 0.9796 - loss: 0.0864 - val_accuracy: 0.9844 - val_loss: 0.0658
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 652ms/step - accuracy: 0.9913 - loss: 0.0371 - val_accuracy: 0.9856 - val_loss: 0.0577
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 670ms/step - accuracy: 0.9966 - loss: 0.0147 - val_accuracy: 0.9880 - val_loss: 0.0615
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 679ms/step - accuracy: 0.9948 - loss: 0.0205 - val_accuracy: 0.9856 - val_loss: 0.0547
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 661ms/step - accuracy: 0.9999 - loss: 0.0029 - val_accuracy: 0.9868 - val_loss: 0.0585
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 673ms/step - accuracy: 0.9996 - loss: 0.0038 - val_accuracy: 0.9880 - val_loss: 0.0591
Epoch 8/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 664ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accu

In [15]:
# Function to predict messages based on model
# (Should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):    
    # Preprocess the input text
    pred_sequence = tokenizer.texts_to_sequences([pred_text])
    pred_sequence_padded = sequence.pad_sequences(pred_sequence, maxlen=MAXLEN)
    
    # Make predictions using the model
    prediction = model.predict(pred_sequence_padded)
    
    # Convert prediction to label
    label = "ham" if prediction < 0.5 else "spam"
    
    return [prediction[0][0], label]

pred_text = "sale today! to stop texts call 98912460324"
prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
[0.9928429, 'spam']


In [16]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
You passed the challenge. Great job!
